In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import random

In [4]:
class Team:
    def __init__(self, name, wins, losses, L5W, L5L, KDA, GPG, CS):
        self.name = name
        self.wins = wins
        self.losses = losses
        self.L5W = L5W
        self.L5L = L5L
        self.KDA = KDA
        self.GPG = GPG
        self.CS = CS
        
    def addWin(self):        
        self.wins += 1
        self.L5W  += 1
        self.L5L  -= 1
        
    def addLoss(self):        
        self.losses += 1
        self.L5W    -= 1
        self.L5L    += 1
        

        

# Simulation

We calculate a teams strength based on 5 metrics:
 - The overall win loss record
 - The last 5 game record to represent recent performance
 - The average team KDA
 - The average gold earned in game (could't find dragons/baron so its the best objective representation I have)
 - The average CS earbed in on game by the whole team
 
In addition we also use weights:
 - The last five games will count a third more than previous game
 - The other attributes will have a weight of 0.25
 

In [5]:
#def play game
def PickWinner(A,B):
    
    #calculate winrate
    WRA = 1.5*(A.wins-A.losses)+(A.L5W-A.L5L)
    WRB = 1.5*(B.wins-B.losses)+(B.L5W-B.L5L)
    
    #normalize WR in %
    NWRA = WRA*100/(WRA+WRB)
    NWRB = WRB*100/(WRA+WRB)
    
    #normalize KDA in %
    NKDAA = A.KDA*100/(A.KDA+B.KDA)
    NKDAB = B.KDA*100/(A.KDA+B.KDA)
    
    #normalize GPG in %
    NGPGA = A.GPG*100/(A.GPG+B.GPG)
    NGPGB = B.GPG*100/(A.GPG+B.GPG)
    
    #normalize CS in %
    NCSA = A.CS*100/(A.CS+B.CS)
    NCSB = B.CS*100/(A.CS+B.CS)
    
    #add weights
    WA = NWRA + 0.25*NKDAA + 0.25*NGPGA + 0.25*NCSA
    WB = NWRB + 0.25*NKDAB + 0.25*NGPGB + 0.25*NCSB
    
    #re normalize in %
    NWA = WA*100/(WA+WB)
    NWB = WB*100/(WA+WB)
    
    #random
    r = random.randint(1,101)
    
    #pick winner
    if(r<NWA):
        A.addWin()
        B.addLoss()
        return A
    else:
        B.addWin()
        A.addLoss()
        return B
    
    

In [6]:
#teams championship count

d = {'G2':[0],'OG':[0],'FNC':[0],'SPY':[0],'VIT':[0],"SK":[0]}
teams = pd.DataFrame(data=d)
teams = teams.rename(index = {0: "WINS"})

In [7]:

def Simulate():
    #teams
    G2  = Team("G2",13,5,1,4,4.06,385.8,1063.92)
    OG = Team("OG",12,6,5,0,4.11,364.8,1090.95)
    FNC  = Team("FNC",11,7,5,0,4.24,366.6,1062.84)
    SPY = Team("SPY",11,7,4,1,4.34,371.6,1235.71)
    VIT = Team("VIT",10,8,1,4,3.24,367.4,1058.37)
    SK = Team("SK",9,9,3,2,2.84,358.4,1064.95)


    #R1G1 3v6 / R1G2 4v5
    R1G1 = PickWinner(FNC,VIT)
    R1G2 = PickWinner(SK,SPY)

    #R2G1 R1G1vR1G2
    R2G1 = PickWinner(R1G1,R1G2)

    #R3G1 1v2
    R3G1 = PickWinner(G2,OG)
    #loser
    if(R3G1.name=="G2"):
        LR3G1 = OG
    else:
        LR3G1 = G2

    #R3G2 R2G1vLR3G1
    R3G2 = PickWinner(R2G1,LR3G1)

    #Final R3G1vR3G2
    Final = PickWinner(R3G1,R3G2)
    return Final.name
    

In [8]:
#simulate x playoffs
for x in range(0,1000000):
    teams[Simulate()]+=1

# Here are our Results

In [9]:
teams

,G2,OG,FNC,SPY,VIT,SK
WINS,283790,401728,143168,134751,16060,20503
